In [45]:
from transformers import BertTokenizerFast, BertForQuestionAnswering
import torch
from torch.utils.data import DataLoader, TensorDataset
from DataGeneration import DataGeneration  # Importing the DataGeneration package

In [46]:
# Load tokenizer and pre-trained BERT model
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')

# Instantiate DataGeneration class
data_generator = DataGeneration()

# Generate data
raw_data_file = "raw_datasets\SUAS-Competition-FALL2023-Final-Report.txt"
train_contexts, train_questions, train_answers = data_generator.generate_data(raw_data_file)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\MarC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\MarC\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\MarC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [47]:
print(train_contexts)

['suas competition software team fall 2023 final report authors marc cruz abdul kalam syed max gross joshua estrada jason mar josh ng ethan tarrer sarkis gafayan rubayet mujahid david jackson status done date sep 21 2023 relative links suas competition software team overview suas competition technical design document suas competition machine learning models 1. introduction two 2. fall 2023 progress three 2.1 timeline three 2.2 recruitment august september four 2.3 learning phase august october four 2.3.1 odlc four 2.3.2 obstacle avoidance five 2.4 odlc design five 2.5 hardware trade studies seven 2.5.1 odlc trade studies seven 2.5.1.1 obc trade study seven 2.5.1.1 camera trade study eight 2.5.2 obstacle avoidance trade study nine 2.6 data collection dataset phase october december ten 2.6.1 standardized object shape ml model ten 2.6.2 standardized object shape alphanumeric color ml model twelve 2.6.3 standardized object alphanumeric ml model thirteen 2.7 modeling phase october february 

In [48]:
print(train_questions)

[['What is 2023?', 'Can you provide details about 2023?', 'Explain the significance of 2023.', 'What are the properties of 2023?'], ['What is yearly?', 'Can you provide details about yearly?', 'Explain the significance of yearly.', 'What are the properties of yearly?'], ['What is semicircle?', 'Can you provide details about semicircle?', 'Explain the significance of semicircle.', 'What are the properties of semicircle?'], ['What is pentagon?', 'Can you provide details about pentagon?', 'Explain the significance of pentagon.', 'What are the properties of pentagon?'], ['What is year?', 'Can you provide details about year?', 'Explain the significance of year.', 'What are the properties of year?'], ['What is mader?', 'Can you provide details about mader?', 'Explain the significance of mader.', 'What are the properties of mader?'], ['What is v8?', 'Can you provide details about v8?', 'Explain the significance of v8.', 'What are the properties of v8?'], ['What is winter?', 'Can you provide d

In [49]:
print(train_answers)

[[{'start_index': 0, 'end_index': 4}, {'start_index': 582, 'end_index': 587}, {'start_index': 26, 'end_index': 30}, {'start_index': 17, 'end_index': 25}, {'start_index': 651, 'end_index': 656}], [{'start_index': 67, 'end_index': 74}, {'start_index': 56, 'end_index': 66}, {'start_index': 44, 'end_index': 55}, {'start_index': 37, 'end_index': 43}, {'start_index': 29, 'end_index': 36}], [{'start_index': 29, 'end_index': 31}, {'start_index': 146, 'end_index': 152}, {'start_index': 137, 'end_index': 145}, {'start_index': 125, 'end_index': 136}, {'start_index': 115, 'end_index': 124}], [{'start_index': 29, 'end_index': 31}, {'start_index': 146, 'end_index': 152}, {'start_index': 137, 'end_index': 145}, {'start_index': 125, 'end_index': 136}, {'start_index': 115, 'end_index': 124}], [{'start_index': 44, 'end_index': 55}, {'start_index': 140, 'end_index': 145}, {'start_index': 132, 'end_index': 136}, {'start_index': 115, 'end_index': 122}, {'start_index': 0, 'end_index': 4}], [{'start_index': 

In [51]:
train_contexts = [str(context) for context in train_contexts]
train_questions = [str(question) for question in train_questions]
# Tokenize inputs
inputs = tokenizer(train_contexts, 
                   train_questions, 
                   padding=True, 
                   truncation=True, 
                   return_tensors='pt',
                   max_length=512, 
                   add_special_tokens=True)

# Convert answers to answer indices
start_positions = []
end_positions = []
for answers in train_answers:
    entity_start_positions = []
    entity_end_positions = []
    for ans in answers:
        start_char = ans['start_index']
        end_char = ans['end_index']
        # Find corresponding token indices using offsets
        start_token = inputs.char_to_token(0, start_char)
        end_token = inputs.char_to_token(0, end_char)
        entity_start_positions.append(start_token)
        entity_end_positions.append(end_token)
    start_positions.append(entity_start_positions)
    end_positions.append(entity_end_positions)

# Filter out None values from start_positions and end_positions
start_positions = [pos for pos in start_positions if pos is not None]
end_positions = [pos for pos in end_positions if pos is not None]

print("Filtered start_positions:", start_positions)
print("Filtered end_positions:", end_positions)

if start_token is None or end_token is None:
    print("Start char:", start_char, "End char:", end_char)
    print("Context:", train_contexts[idx])
    print("Question:", train_questions[idx])

# Convert to PyTorch tensors
start_positions = torch.tensor(start_positions)
end_positions = torch.tensor(end_positions)

# Define optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
loss_fn = torch.nn.CrossEntropyLoss()

# Training loop
num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids, token_type_ids, attention_mask, start_positions, end_positions = batch
        outputs = model(input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask,
                        start_positions=start_positions, end_positions=end_positions)
        loss = loss_fn(outputs.start_logits, start_positions.view(-1)) + loss_fn(outputs.end_logits, end_positions.view(-1))
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch + 1}, Loss: {total_loss}')

# Save the trained model
model.save_pretrained('bert_qa_model')

Filtered start_positions: [[1, 129, 5, 4, 153], [13, 11, 9, 7, 5], [5, 29, None, 24, 22], [5, 29, None, 24, 22], [9, 27, 26, 22, 1], [26, 25, 24, 22, 21], [None, 60, 54, 41, 41], [13, None, None, 21, 20], [6, None, 15, 13, 12], [3, 4, 53, 4, 40], [None, 13, 33, None, None], [4, 3, 24, None, 59], [20, 9, 4, 61, 60], [10, 12, 38, 35, 30], [10, 12, 38, 35, 30], [13, None, None, 21, 20], [1, 129, 3, 5, 4]]
Filtered end_positions: [[None, None, None, None, None], [14, None, 11, 9, 7], [6, 31, 29, 26, 24], [6, 31, 29, 26, 24], [11, 29, 26, 23, None], [28, 26, None, None, 22], [56, 60, None, 41, None], [14, 13, 25, 22, 21], [7, 19, 18, None, 13], [None, 4, 53, 4, 41], [30, 14, 35, 33, 28], [4, 4, None, 61, 59], [21, 10, 6, 62, 61], [11, 12, 40, 38, 31], [11, 12, 40, 38, 31], [14, 13, 25, 22, 21], [None, None, None, None, None]]
Start char: 17 End char: 25
Context: suas competition software team fall 2023 final report authors marc cruz abdul kalam syed max gross joshua estrada jason mar josh n

RuntimeError: Could not infer dtype of NoneType